<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/rivaasrasr-wordboosting/nvidia_logo.png" style="width: 90px; float: right;">

# How do I boost specific words at runtime with word boosting?
# Take Mandarin (ZH) as an example

This tutorial walks you through some of the advanced features for customization of Riva Speech Skills ASR Services at runtime with word boosting.

## 0. NVIDIA Riva Overview

NVIDIA Riva is a GPU-accelerated SDK for building Speech AI applications that are customized for your use case and deliver real-time performance. <br/>
Riva offers a rich set of speech and natural language understanding services such as:

- Automated speech recognition (ASR)
- Text-to-Speech synthesis (TTS)
- A collection of natural language processing (NLP) services, such as named entity recognition (NER), punctuation, intent classification.

In this tutorial, we will customize Riva ASR to boost specific words at runtime with word boosting. <br> 
To understand the basics of Riva ASR APIs, refer to [Getting started with Riva ASR in Python](https://github.com/nvidia-riva/tutorials/tree/stable/asr-python-basics.ipynb). <br>

For more information about Riva, refer to the [Riva developer documentation](https://developer.nvidia.com/riva).

## 1. Word boosting with Riva ASR APIs

Word boosting is one of the customizations Riva offers. It allows you to bias the ASR engine to recognize particular words of interest at request time by giving them a higher score when decoding the output of the acoustic model.  

Now, let's use word boosting with Riva APIs for some sample audio clips with an OOTB (out-of-the-box) Mandarin pipeline.

### 1.1 Requirements and setup

#### 1.1.1 Start the Riva Speech Skills server.  
Follow the instructions in the [Riva Skills Quick Start Guide](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html#) to deploy OOTB ASR models on the Riva Speech Skills server before running this tutorial. 

HINTS: 
To deploy a Mandarin Model, you have to do set the following parameters in `riva_quickstart_*/config.sh`:
 * language_code=("zh-CN")
 * service_enabled_asr=true
 * service_enabled_nlp=false
 * service_enabled_tts=false
 * service_enabled_nmt=false

#### 1.1.2 Install the Riva Client library.  
Follow the steps in the [Requirements and setup for the Riva Client](https://github.com/nvidia-riva/tutorials#running-the-riva-client) to install the Riva Client library.

In [ ]:
!pip3 install nvidia-riva-client

#### 1.1.3 Import the Riva client libraries

Let's import some of the required libraries, including the Riva Client libraries.

In [ ]:
import io
import IPython.display as ipd
import grpc

import riva.client

#### 1.1.4 Create a Riva client and connect to the Riva Speech API server

The following URI assumes a local deployment of the Riva Speech API server is on the default port. In case the server deployment is on a different host or via a Helm chart on Kubernetes, use an appropriate URI.

In [ ]:
auth = riva.client.Auth(uri='localhost:50051')

riva_asr = riva.client.ASRService(auth)

### 2. Example: how to use word boosting
Here, we give one example to show how to use word boosting for Mandarin ASR model. 

#### 2.1 ASR inference without word boosting
First, let's run ASR on our sample audio clip without word boosting.

In [ ]:
# Load a sample audio file 
path = 'audio_sample/zh_wordboosting.wav'
with io.open(path, 'rb') as fh:
    content = fh.read()
ipd.Audio(path)

In [ ]:
# Creating RecognitionConfig
config = riva.client.RecognitionConfig(
    sample_rate_hertz=0,
    language_code="zh-CN",
    max_alternatives=1,
    audio_channel_count=1,
    verbatim_transcripts=False
)

response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript without Word Boosting:", asr_best_transcript)

As you can see, ASR is having a hard time recognizing specific names like `李佳琦`. <br>

Now, let's use word boosting to try to improve ASR for these specific terms.

#### 2.2 ASR inference with word boosting

Let's look at how to add the boosted words to `RecognitionConfig` with `SpeechContext`. (For more information about `SpeechContext`, refer to the docs [here](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/reference/protos/protos.html#_CPPv413SpeechContext)). The simplest way to add word boosting is to use function
[riva.client.add_word_boosting_to_config()](https://github.com/nvidia-riva/python-clients/blob/928c63273176a939500e01ce176c463f1606a1ff/riva_api/asr.py#L78).

In [ ]:
# Creating SpeechContext for word boosting
boosted_lm_words = ["李 佳 琦 "]
boosted_lm_score = 20.0
riva.client.add_word_boosting_to_config(config, boosted_lm_words, boosted_lm_score)

# ASR Inference call with Recognize 
response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript with Word Boosting:", asr_best_transcript)

As you can see, with word boosting, ASR is able to correctly transcribe the specific people's name like `李佳琦`.

* `Boost Score`: The recommended range for the boost score is 20 to 100. The higher the boost score, the more biased the ASR engine is towards this word.  
* `OOV Word Boosting`: OOV words can also be word boosted; in the exact same way as in-vocabulary words, as described above.

### 3. Advanced word boosting
#### 3.1 Boosting different words at different levels
With Riva ASR, we can also have different boost values for different words. See the next example: 

In [ ]:
# Load a sample audio file from local disk
# This example uses a .wav file with LINEAR_PCM encoding.
path = "audio_sample/zh_wordboosting2.wav"
with io.open(path, 'rb') as fh:
    content = fh.read()
ipd.Audio(path)

In [ ]:
# Creating RecognitionConfig
config = riva.client.RecognitionConfig(
    language_code="zh-CN",
    max_alternatives=1,
    audio_channel_count=1,
)

response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript without Word Boosting:", asr_best_transcript)

We can set different Boost Score for `五G` (10.0) and `四G` (20.0). If you have other words to be boosted, try with `add_word_boosting_to_config` function and with your own words!

In [ ]:
# Creating RecognitionConfig
config = riva.client.RecognitionConfig(
  language_code="zh-CN",
  max_alternatives=1,
  audio_channel_count = 1
)
riva.client.add_word_boosting_to_config(config, ["五 G "], 10.)
riva.client.add_word_boosting_to_config(config, ["四 G "], 20.)

# ASR Inference call with Recognize 
response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript with Word Boosting:", asr_best_transcript)

#### 3.2 Negative word boosting for undesired words
We can even use word boosting to discourage prediction of some words, by using negative boost scores.  

Let's load a sample audio file and get the transcription results from it without any word boosting

In [ ]:
# Load a sample audio file from local disk
# This example uses a .wav file with LINEAR_PCM encoding.
path = "audio_sample/zh_wordboosting.wav"
with io.open(path, 'rb') as fh:
    content = fh.read()
ipd.Audio(path)

In [ ]:
# Creating RecognitionConfig
config = riva.client.RecognitionConfig(
  language_code="zh-CN",
  max_alternatives=1,
  audio_channel_count = 1
)

# ASR Inference call with Recognize 
response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript without Word Boosting:", asr_best_transcript)

Now, let us get the transcription results with negative word boosting for the word `她`.

In [ ]:
# Creating RecognitionConfig
config = riva.client.RecognitionConfig(
  language_code="zh-CN",
  max_alternatives=1,
  audio_channel_count = 1
)
riva.client.add_word_boosting_to_config(config, ["她"], -100.)

# ASR Inference call with Recognize 
response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript with Negative Word Boosting:", asr_best_transcript)

As you can see, the word `她` was not predicted this time. Also note that in this case, `他` was predicted in its place. 

#### 3.3 Combine negative and positive word boosting
Let us combine the negative word boosting from above example, with positive word boosting to predict the word `李佳琦`. 

We perform positive word boosting, as we did in earlier examples, for word `李佳琦`. Note that we are using the same `config` instance, that we created in the above example for negative word boosting for `她`. So it already has the `SpeechContext` instance. So, now, we just need to add a `SpeechContext` instance for positive word boosting for `middle`.

In [ ]:
# Creating SpeechContext for Word Boosting
positive_boosted_lm_word = "李 佳 琦 "
positive_boosted_lm_score = 20.0
riva.client.add_word_boosting_to_config(config, [positive_boosted_lm_word], positive_boosted_lm_score)

# ASR Inference call with Recognize 
response = riva_asr.offline_recognize(content, config)
asr_best_transcript = response.results[0].alternatives[0].transcript
print("ASR Transcript with Negative and Positive Word Boosting:", asr_best_transcript)

As you can see, in this example, we were able to generate the transcript by combining negative and positive word boosting.

Note:

- There is no limit to the number of words that can be boosted. You should see no impact on latency for all requests, even for ~100 boosted words, except for the first request, which is expected.
- Boosting phrases or a combination of words is not yet fully supported (but do work). We will revisit finalizing this support in an upcoming release.
- By default, no words are boosted on the server side. Only words passed by the client are boosted.

Information about word boosting can also be found in the documentation [here](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/asr/asr-customizing.html#word-boosting). 

## 4. Go deeper into Riva capabilities


### Additional Riva Tutorials

Checkout more Riva tutorials [here](https://github.com/nvidia-riva/tutorials) to understand how to use some of the advanced features of Riva ASR, including customizing ASR for your specific needs.


### Sample Applications

Riva comes with various sample applications. They demonstrate how to use the APIs to build various applications. Refer to [Riva Sampple Apps](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/samples/index.html) for more information.  


### Additional Resources

For more information about each of the Riva APIs and their functionalities, refer to the [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/reference/protos/protos.html).